
#IMPORTING LIBRARIES
This code performs text classification using NLTK and scikit-learn libraries, including data preprocessing, model training, and evaluation steps with various classifiers. It downloads NLTK resources, imports necessary modules, ignores warnings, and defines machine learning models for classification tasks.

In [ ]:
import nltk
nltk.download('all')
import random
import json
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

#CHATBOT INTENT DATA
This JSON data represents intents for a chatbot, including tags, patterns, responses, and context. Each intent corresponds to a specific query or topic that the chatbot can respond to. The patterns are examples of user queries, responses are the bot's replies, and context_set can be used for maintaining conversation context. This data serves as a training dataset for the chatbot to understand and respond to user queries appropriately.

In [ ]:
intents = {
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hi", "Hello", "Hey", "Good day", "How are you?"],
      "responses": ["Hello!", "Good to see you!", "Hi there, how can I help?"],
      "context_set": ""
    },
    {
      "tag": "farewell",
      "patterns": ["Goodbye", "Bye", "See you later", "Talk to you later"],
      "responses": ["Sad to see you go :(", "Goodbye!", "Come back soon!"],
      "context_set": ""
    },
    {
      "tag": "creator",
      "patterns": ["Who created you?", "Who is your developer?", "Who made you?","Who invented you?"],
      "responses": ["I was created by Veludandi Harini."],
      "context_set": ""
    },
    {
      "tag": "identity",
      "patterns": ["What is your name?", "What should I call you?", "Who are you?"],
      "responses": ["You can call me harini. I'm a Chatbot."],
      "context_set": ""
    },
    {
      "tag": "hours",
      "patterns": ["College Timings","What are the college timings?", "When is the college open?", "What are your hours of operation?"],
      "responses": ["The college is open from 9:10 am to 4:00 pm, Monday to Friday."],
      "context_set": ""
    },
    {
      "tag": "contact",
      "patterns": ["How can I contact the college?", "What is the college telephone number?", "Can I get your contact number?","Contact details please!"],
      "responses": ["You can contact the college at 9618358985"],
      "context_set": ""
    },
    {
      "tag": "courses",
      "patterns": ["What courses are offered in the college?", "Can you tell me about the available courses?", "What are the branches in the college?"],
      "responses": ["The college offers courses in Artificial Intellegence and Machine Learning, Cyber Security, Data Science, Artifficial Intellegence and Data Science, Information Technology, Computer Engineering, Mechanical Engineering, Chemical Engineering, and Civil Engineering."],
      "context_set": ""
    },
    {
      "tag": "fees",
      "patterns": ["How much is the college fee?", "Tell me about the fees", "What are the hostel fees?"],
      "responses": ["For detailed fee information, please visit our college website."],
      "context_set": ""
    },
    {
      "tag": "location",
      "patterns": ["Where is the college located?", "What is the college address?", "How can I reach the college?"],
      "responses": ["The college is located at bollikunta, Warangal district. You can find the location on Google Maps."],
      "context_set": ""
    },
    {
      "tag": "hostel",
      "patterns": ["Does the college provide hostel facilities?", "Where is the hostel located?", "What are the hostel fees?"],
      "responses": ["Yes, College provides hostel facilities. For detailed information about the hostel, please visit our college website."],
      "context_set": ""
    },
    {
      "tag": "events",
      "patterns": ["What events are organized in the college?", "Are there any upcoming events?", "Tell me about the college events."],
      "responses": ["Events will be displayed in the college website. For information about the events, please visit our college website."],
      "context_set": ""
    },
    {
      "tag": "admission",
      "patterns": ["What is the admission process?", "How can I take admission in the college?", "Tell me about the admission criteria."],
      "responses": ["You have to write Eamcet exam as a part of admission process. For detailed admission information, please visit our college website."],
      "context_set": ""
    },
    {
      "tag": "library",
      "patterns": ["Does the college have a library?", "Where is the library located?", "What are the library timings?"],
      "responses": ["Yes, the college has a library. The library is located near A-Block and w-block in the university. The timings are from 8am to 9pm, Monday to Friday."],
      "context_set": ""
    },
    {
      "tag": "facilities",
      "patterns": ["What facilities are available in the college?", "Tell me about the college facilities.", "Do you have a sports complex?"],
      "responses": ["The college provides various facilities including a sports complex, labs, cafeteria, Different branches of clubs and a library."],
      "context_set": ""
    },
    {
      "tag": "placement",
      "patterns": ["How are the placements in the college?", "Tell me about the college placements.", "What is the placement record?"],
      "responses": ["The college has a good placement record. Many reputed companies visit our campus for recruitment."],
      "context_set": ""
    },
    {
      "tag": "school_choice",
      "patterns": ["Why did you choose this school?", "What made you decide to attend this college?"],
      "responses": ["I chose this college because of its strong academic reputation, excellent faculty, and a wide range of courses offered."],
      "context_set": ""
    },
    {
      "tag": "happiness",
      "patterns": ["Are you happy here?", "Do you enjoy being at this college?"],
      "responses": ["As an AI, I don't have emotions, but I'm here to assist and provide information about the college."],
      "context_set": ""
    },
    {
      "tag": "strengths",
      "patterns": ["What are your school's strengths?", "What makes your college stand out?"],
      "responses": ["Our college excels in providing quality education, fostering a supportive learning environment, and offering a wide range of extracurricular activities."],
      "context_set": ""
    },
    {
      "tag": "complaints",
      "patterns": ["What's your #1 complaint about your school?", "What do you think needs improvement in your college?"],
      "responses": ["While my purpose is to provide information, some students may have different opinions. It's always good to provide feedback for continuous improvement."],
      "context_set": ""
    },
    {
      "tag": "improvements",
      "patterns": ["What else could stand to be improved?", "In which areas do you think your college can improve?"],
      "responses": ["Continuous improvement is essential for any institution. Areas that can be considered for improvement include infrastructure, facilities, and student support services."],
      "context_set": ""
    },
    {
      "tag": "accessibility",
      "patterns": ["How accessible are administrators, registrars, financial aid officers, etc.?", "Can students easily reach out to college staff when needed?"],
      "responses": ["The college administration strives to maintain accessibility for students. Office hours and contact information for various staff members are available for student assistance."],
      "context_set": ""
    },
    {
      "tag": "academics_majors",
      "patterns": ["What majors are popular?", "Which fields of study have a high enrollment?", "What are the most sought-after majors in your college?"],
      "responses": ["Popular majors in our college include Computer Science, Business Administration, Psychology, Engineering, and Biology."],
      "context_set": ""
    },
    {
      "tag": "academics_reputation",
      "patterns": ["What departments or programs have the best reputations?", "Which academic programs are well-regarded?", "Are there any standout departments in your college?"],
      "responses": ["Our college has well-regarded departments in Computer Science, Engineering, and Business Administration. These programs have a strong reputation for academic excellence."],
      "context_set": ""
    },
    {
      "tag": "favorite_class",
      "patterns": ["What's your favorite class?", "Which class do you enjoy the most?", "Do you have a favorite subject?"],
      "responses": ["As an AI, I don't have personal preferences. However, students often enjoy classes that align with their interests and career goals."],
      "context_set": ""
    },
    {
      "tag": "professors_teachers",
      "patterns": ["Are your professors good teachers?", "How is the quality of teaching at your college?", "Do you have experienced faculty?"],
      "responses": ["Our college prides itself on having experienced and dedicated faculty members who are committed to providing quality education."],
      "context_set": ""
    },
    {
      "tag": "professors_office_hours",
      "patterns": ["Do your professors hold office hours, and will they meet with you outside of class?", "Are professors available for one-on-one meetings?", "Can students easily meet with professors?"],
      "responses": ["Most professors at our college hold regular office hours and are available for one-on-one meetings to address student queries and provide additional guidance."],
      "context_set": ""
    },
    {
      "tag": "professors_vs_TAs",
      "patterns": ["Are most of your classes taught by professors or teaching assistants?", "Who primarily teaches the classes?", "Do you have a lot of classes taught by TAs?"],
      "responses": ["The majority of classes at our college are taught by professors. However, in some cases, teaching assistants may assist with course instruction or hold discussion sessions."],
      "context_set": ""
    },
    {
      "tag": "first_choice_classes",
      "patterns": ["Were you able to take most of your first-choice classes?", "Do students generally get their preferred classes?", "How likely are you to get into your desired classes?"],
      "responses": ["While course availability can vary, our college aims to accommodate students' preferences as much as possible. Course registration is conducted on a first-come, first-served basis."],
      "context_set": ""
    },
    {
      "tag": "class_format",
      "patterns": ["Are your classes lecture-based or discussion-based?", "How are the classes structured?", "What's the typical format of a class?"],
      "responses": ["The class format can vary depending on the subject and the instructor's teaching style. Some classes may be lecture-based, while others may involve more interactive discussions and group activities."],
      "context_set": ""
    },
    {
      "tag": "reading_writing",
      "patterns": ["How much reading and writing is required in your courses?", "Are there a lot of assignments involving reading and writing?", "Is there a heavy emphasis on written work?"],
      "responses": ["The amount of reading and writing required in courses can vary. Generally, students can expect a balanced mix of reading assignments, written projects, and exams."],
      "context_set": ""
    },
    {
      "tag": "collaborative_work",
      "patterns": ["How often are collaborative work or group presentations required?", "Do you have a lot of group projects?", "Is there an emphasis on teamwork?"],
      "responses": ["Collaborative work and group presentations are part of the learning experience at our college. Students often engage in group projects to enhance teamwork and develop important skills."],
      "context_set": ""
    },
    {
      "tag": "academic_advising",
      "patterns": ["How satisfied are you with academic advising?", "Is the academic advising program helpful?", "Do advisors provide guidance for course selection?"],
      "responses": ["Academic advising plays a crucial role in guiding students' academic journeys. Our college has dedicated advisors who provide assistance with course selection, degree planning, and academic support."],
      "context_set": ""
    },
    {
      "tag": "tutoring_programs",
      "patterns": ["Do students use any on-campus tutoring programs or writing centers?", "Are there resources available for academic support?", "How can I get tutoring assistance?"],
      "responses": ["Our college provides on-campus tutoring programs and writing centers to support students in their academic endeavors. These resources can help students with subject-specific assistance and writing skills."],
      "context_set": ""
    },
    {
      "tag": "class_size",
      "patterns": ["How big are your classes?", "Are the class sizes large or small?", "Do you have small class sizes?"],
      "responses": ["Class sizes can vary depending on the course and program. While some classes may have a larger enrollment, others, especially upper-level courses, tend to have smaller class sizes for more personalized attention."],
      "context_set": ""
    },
    {
      "tag": "independent_study",
      "patterns": ["Does your major require an independent study or capstone project?", "Are there opportunities for independent research?", "Is there a culminating project for your major?"],
      "responses": ["Many majors at our college offer opportunities for independent study or capstone projects. These experiences allow students to delve deeper into their fields of interest and apply their knowledge."],
      "context_set": ""
    },
    {
      "tag": "professor_research",
      "patterns": ["Are professors available for research with students?", "Can students engage in research projects with faculty?", "How can I get involved in research at your college?"],
      "responses": ["Faculty members at our college actively engage in research, and students often have opportunities to collaborate on research projects. You can reach out to professors in your field of interest to inquire about research opportunities."],
      "context_set": ""
    },
    {
      "tag": "study_abroad",
      "patterns": ["Is it popular to study abroad?", "Are there study abroad programs available?", "Can students spend a semester abroad?"],
      "responses": ["Studying abroad is a popular option for many students at our college. We offer various study abroad programs and opportunities to experience different cultures and gain international exposure."],
      "context_set": ""
    },
    {
      "tag": "first_year_experience",
      "patterns": ["What's it like to be a first-year student here?", "Can you describe the first-year experience?", "What should I expect as a freshman?"],
      "responses": ["As a first-year student, you can expect a supportive and welcoming environment. Orientation programs, peer mentoring, and academic support services are available to help you transition smoothly into college life."],
      "context_set": ""
    },
    {
      "tag": "typical_day",
      "patterns": ["What's a typical day like?", "Can you describe a day in the life of a student?", "What's the daily routine at your college?"],
      "responses": ["A typical day as a student involves attending classes, engaging in study sessions, participating in extracurricular activities, and utilizing campus resources. Each student's daily routine may vary based on their schedule and interests."],
      "context_set": ""
    },
    {
      "tag": "study_time",
      "patterns": ["How much time do students spend studying per week?", "Is there a lot of studying required?", "What's the average study time for students?"],
      "responses": ["The amount of time students spend studying can vary depending on their course load, study habits, and personal preferences. On average, students dedicate several hours per week to studying."],
      "context_set": ""
    },
    {
      "tag": "non_class_activities",
      "patterns": ["What do you do when you're not in class?", "How do students spend their free time?", "Are there activities to do outside of class?"],
      "responses": ["Outside of class, students engage in various activities such as participating in clubs and organizations, attending campus events, exploring the surrounding community, and spending time with friends."],
      "context_set": ""
    },
    {
      "tag": "weekend_activities",
      "patterns": ["What do you do on the weekends?", "Are there weekend activities or events?", "How do students spend their weekends?"],
      "responses": ["On weekends, students often take part in campus events, recreational activities, social gatherings, and may also use the time for relaxation and personal pursuits."],
      "context_set": ""
    },
    {
      "tag": "social_scene",
      "patterns": ["What is the social scene like?", "How would you describe the social life on campus?", "Are there opportunities to meet new people?"],
      "responses": ["The social scene on campus is vibrant and diverse. There are plenty of opportunities to meet new people, join clubs and organizations, attend social events, and build lifelong friendships."],
      "context_set": ""
    },
    {
      "tag": "hometown_activities",
      "patterns": ["What kinds of things are there to do in your school's hometown?", "Are there recreational activities in the local area?", "What are some popular attractions near the college?"],
      "responses": ["Our school's hometown offers a range of recreational activities, cultural events, shopping centers, restaurants, and natural attractions. You can explore local parks, museums, theaters, and enjoy the local cuisine."],
      "context_set": ""
    },
    {
      "tag": "food",
      "patterns": ["How's the food?", "What is the quality of the food on campus?", "Are there good dining options?"],
      "responses": ["The college provides a variety of dining options on campus, including cafeterias, food courts, and specialty restaurants. The quality of food is generally satisfactory, with options to cater to different dietary preferences."],
      "context_set": ""
    },
    {
      "tag": "on_campus_housing",
      "patterns": ["Do most students live on-campus?", "Are there on-campus housing options?", "What percentage of students live in dorms?"],
      "responses": ["While the availability of on-campus housing may vary, many students choose to live in dorms or residence halls provided by the college. Off-campus housing options are also available for students."],
      "context_set": ""
    },
    {
      "tag": "dorm_quality",
      "patterns": ["How are the dorms?", "Are the dormitories comfortable?", "What's the quality of on-campus housing?"],
      "responses": ["The quality of dormitories varies based on the specific residence halls. Generally, the college strives to provide comfortable and convenient living spaces with amenities that cater to students' needs."],
      "context_set": ""
    },
    {
      "tag": "popular_clubs",
      "patterns": ["Which clubs and student organizations are popular?", "Are there any standout clubs on campus?", "What are some of the most active student organizations?"],
      "responses": ["Our college has a wide range of clubs and student organizations catering to diverse interests. Popular clubs include academic organizations, cultural clubs, sports teams, community service groups, and more."],
      "context_set": ""
    },
    {
      "tag": "fraternities_sororities",
      "patterns": ["Do lots of students belong to fraternities or sororities?", "Are there many Greek organizations on campus?", "What's the Greek life like at your college?"],
      "responses": ["While the presence and prominence of fraternities and sororities can vary among colleges, our college does have a Greek life community. Participation in these organizations is a personal choice for interested students."],
      "context_set": ""
    },
    {
      "tag": "favorite_place_on_campus",
      "patterns": ["What's your favorite place on campus?", "Which location do you enjoy the most?", "Where do students like to hang out?"],
      "responses": ["As an AI, I don't have personal preferences. However, many students enjoy spending time in common areas, libraries, student centers, outdoor spaces, and campus cafeterias."],
      "context_set": ""
    },
    {
      "tag": "study_locations",
      "patterns": ["Where do you like to study?", "What are some good study spots?", "Are there quiet places to study on campus?"],
      "responses": ["There are several study spots on campus, including libraries, study lounges, coffee shops, and outdoor seating areas. It's important to find a location that suits your study preferences and helps you focus."],
      "context_set": ""
    },
    {
      "tag": "campus_facilities",
      "patterns": ["What are the facilities like (science labs, libraries, theaters, gyms, etc.)?", "Are the campus facilities well-maintained?", "Does the college have modern facilities?"],
      "responses": ["Our college prides itself on maintaining well-equipped facilities to support students' academic and extracurricular activities. This includes modern science labs, libraries with extensive resources, theaters, and well-equipped gyms."],
      "context_set": ""
    },
    {
      "tag": "computer_labs",
      "patterns": ["Are there enough computer labs?", "How accessible are the computer labs?", "Is there a shortage of computer lab facilities?"],
      "responses": ["Our college provides an ample number of computer labs across campus. These labs are accessible to students and equipped with the necessary technology and software needed for academic work."],
      "context_set": ""
    },
    {
      "tag": "campus_wifi",
      "patterns": ["How is the WiFi on campus?", "Is the campus WiFi reliable?", "Are there any connectivity issues with the WiFi?"],
      "responses": ["The campus WiFi network is designed to provide reliable internet connectivity to students, faculty, and staff. However, occasional connectivity issues may arise, which are usually promptly addressed by the IT department."],
      "context_set": ""
    },
    {
      "tag": "campus_navigation",
      "patterns": ["Is it easy to get around campus?", "Are the campus buildings easy to navigate?", "Is the campus layout student-friendly?"],
      "responses": ["The campus is designed to be student-friendly, and campus buildings are typically well-marked and easy to navigate. Campus maps, signage, and digital resources are available to help students find their way around."],
      "context_set": ""
    },
    {
      "tag": "sports_popularity",
      "patterns": ["Are sports popular?", "Do many students participate in sports?", "What's the athletic culture like at your college?"],
      "responses": ["Sports are popular at our college, and many students participate in various athletic activities. We have a range of sports teams, intramural sports, and fitness programs to cater to different interests and skill levels."],
      "context_set": ""
    },
    {
      "tag": "school_spirit",
      "patterns": ["Is there a lot of school spirit?", "Do students show a strong sense of school pride?", "Are there spirited events or traditions?"],
      "responses": ["Our college has a vibrant school spirit, and students often show a strong sense of pride in their institution. There are spirited events, traditions, and opportunities to engage with the college community and showcase school pride."],
      "context_set": ""
    },
    {
      "tag": "fellow_students",
      "patterns": ["How would you describe your fellow students?", "What are the students like?", "Is there a sense of community among students?"],
      "responses": ["The student body is diverse and inclusive, with individuals from various backgrounds, interests, and experiences. Students generally foster a sense of community and support each other's academic and personal growth."],
      "context_set": ""
    },
    {
      "tag": "friendly_students",
      "patterns": ["Are the students here friendly?", "Is it easy to make friends?", "Do students interact with each other?"],
      "responses": ["Yes, the students here are generally friendly and welcoming. The college environment provides ample opportunities to meet new people, make friends, and form meaningful connections both inside and outside the classroom."],
      "context_set": ""
    },
    {
      "tag": "student_body_unique",
      "patterns": ["What makes the student body unique?", "Is there anything special about the student population?", "What sets the students apart at your college?"],
      "responses": ["The student body at our college is unique in its diversity of backgrounds, perspectives, talents, and passions. This diversity fosters a rich learning environment and encourages students to embrace different cultures and viewpoints."],
      "context_set": ""
    },
    {
      "tag": "campus_diversity",
      "patterns": ["Is there diversity on campus?", "Are there students from different cultural backgrounds?", "Does the college promote diversity and inclusion?"],
      "responses": ["Yes, our college values diversity and strives to create an inclusive campus community. Students from different cultural backgrounds, ethnicities, and countries contribute to the diverse fabric of our college community."],
      "context_set": ""
    },
    {
      "tag": "international_students",
      "patterns": ["Are there many students from other countries?", "Do you have a significant international student population?", "How diverse is the international representation at your college?"],
      "responses": ["Our college welcomes students from around the world, and we have a diverse international student population. International students contribute to the cultural exchange and global perspectives within the college community."],
      "context_set": ""
    },
    {
      "tag": "interactions_among_students",
      "patterns": ["Do students of different races and classes interact easily?", "Is there interaction and socialization among diverse groups?", "Do students embrace diversity and engage in cross-cultural interactions?"],
      "responses": ["Students at our college have opportunities to interact and socialize with peers from different races, classes, and cultural backgrounds. The college promotes a welcoming and inclusive environment that encourages cross-cultural interactions and understanding."],
      "context_set": ""
    },
    {
      "tag": "student_cliques",
      "patterns": ["Are students cliquish?", "Is there a presence of cliques on campus?", "Do students segregate into exclusive groups?"],
      "responses": ["While some social groups may naturally form based on shared interests and affiliations, our college encourages inclusivity and fosters an environment where students from different backgrounds can mingle and form connections beyond cliques."],
      "context_set": ""
    },
    {
      "tag": "internship_opportunities",
      "patterns": ["Are internships available? How do you find them?", "What internship opportunities are there?", "How does the college support students in finding internships?"],
      "responses": ["Internship opportunities are available for students, and the college often provides resources and support to help students find and secure internships. The college's career services office and faculty advisors can offer guidance in exploring and applying for internships."],
      "context_set": ""
    },
    {
      "tag": "career_services",
      "patterns": ["Is Career Services helpful?", "What services does the career center provide?", "How effective is the college's career guidance?"],
      "responses": ["Career Services at our college offers various resources and assistance to help students with career exploration, resume building, job search strategies, interview preparation, and more. Many students find the career guidance helpful in their professional development."],
      "context_set": ""
    },
    {
      "tag": "mentorship_program",
      "patterns": ["Do you have a mentor?", "Is there a mentorship program?", "How can students connect with mentors?"],
      "responses": ["Our college may have a mentorship program or opportunities for students to connect with mentors. These programs facilitate valuable guidance and support from experienced individuals who can provide insights and advice on academic and career paths."],
      "context_set": ""
    },
    {
      "tag": "co-op_program",
      "patterns": ["Does your school have a co-op program?", "Are there cooperative education opportunities?", "How does the college support students in co-op experiences?"],
      "responses": ["Some colleges offer co-op programs where students can gain practical work experience related to their field of study while earning academic credit. If our college has a co-op program, students can explore such opportunities with the support of the college's career services or cooperative education office."],
      "context_set": ""
    },
    {
      "tag": "leadership_opportunities",
      "patterns": ["What options exist for leadership skills on campus?", "Are there leadership programs or positions?", "How can students develop leadership skills?"],
      "responses": ["Our college offers various opportunities for students to develop leadership skills. Students can join student government, clubs, or organizations that provide leadership positions, participate in leadership workshops or programs, and engage in community service to foster leadership qualities."],
      "context_set": ""
    },
    {
      "tag": "service_learning",
      "patterns": ["What options exist for service learning in the community?", "Are there community service programs?", "How can students get involved in community service?"],
      "responses": ["Our college often encourages service learning and community engagement. Students can get involved in community service programs, volunteer with local organizations, participate in service-learning courses, or join campus initiatives dedicated to making a positive impact in the community."],
      "context_set": ""
    },
    {
      "tag": "employer_recruitment",
      "patterns": ["Do employers recruit students on campus?", "Are there career fairs or job fairs?", "How does the college connect students with potential employers?"],
      "responses": ["Employer recruitment is a common practice at many colleges. Career fairs, job fairs, and networking events are organized to connect students with potential employers. The college's career services office often plays a role in facilitating these connections."],
      "context_set": ""
    },
    {
      "tag": "summer_jobs",
      "patterns": ["How easy is it to find summer jobs and other kinds of work through your school?", "Are there resources for finding summer employment?", "Does the college assist with summer job opportunities?"],
      "responses": ["Our college may provide resources and support to help students find summer jobs and other types of employment. The career services office, online job boards, and networking events can be valuable in exploring and securing summer job opportunities."],
      "context_set": ""
    },
    {
      "tag": "notable_graduates",
      "patterns": ["Who are some of the notable graduates from your college?", "Can you mention any famous alumni?", "Which successful individuals attended your college?"],
      "responses": ["Our college has a distinguished list of notable graduates and successful alumni in various fields. Some may include renowned professionals, entrepreneurs, artists, scientists, and leaders who have made significant contributions to their respective fields."],
      "context_set": ""
    },
    {
      "tag": "alumni_association",
      "patterns": ["How visible is your school's alumni association on campus?", "Does the college have an active alumni network?", "Are there opportunities to connect with alumni?"],
      "responses": ["Many colleges have active alumni associations that offer networking opportunities, mentorship programs, and career support for current students. The visibility and engagement of the alumni association can vary, but efforts are often made to connect students with alumni."],
      "context_set": ""
    },
    {
      "tag": "hands-on_experiences",
      "patterns": ["What kinds of hands-on or practical experiences have you had in and outside of the classroom?", "Are there opportunities for practical learning?", "How does the college emphasize experiential learning?"],
      "responses": ["Colleges often provide opportunities for hands-on and practical experiences both inside and outside the classroom. These may include internships, research projects, fieldwork, service-learning, study abroad programs, and collaborative projects that enhance the learning process and offer real-world application of knowledge."],
      "context_set": ""
    }
  ]
}

#Synonym Replacement Augmentation Function
This function, synonym_replacement, takes a list of tokens and a limit as input parameters. It generates augmented sentences by replacing each token in the input with a synonym obtained from WordNet. The limit parameter controls the maximum number of synonyms to be considered for replacement. It returns a list of augmented sentences where each token in the original sentence has been replaced with a synonym, resulting in a set of varied sentences for data augmentation purposes.

In [ ]:
def synonym_replacement(tokens, limit):
    augmented_sentences = []
    for i in range(len(tokens)):
        synonyms = []
        for syn in wordnet.synsets(tokens[i]):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if len(synonyms) > 0:
            num_augmentations = min(limit, len(synonyms))
            sampled_synonyms = random.sample(synonyms, num_augmentations)
            for synonym in sampled_synonyms:
                augmented_tokens = tokens[:i] + [synonym] + tokens[i+1:]
                augmented_sentences.append(' '.join(augmented_tokens))
    return augmented_sentences

#Data Preparation for Text Classification
This code snippet prepares the data for text classification by iterating through the intents defined earlier. For each intent, it tokenizes the patterns, filters out stopwords, and lemmatizes the tokens. Then, it augments the data by generating synonym replacements using the synonym_replacement function defined earlier, up to a limit defined by limit_per_tag. Finally, it appends the filtered and augmented sentences to text_data and their corresponding labels to labels for training the classification model. This process ensures a diverse and augmented dataset for robust text classification.

In [ ]:
text_data = []
labels = []
stopwords = set(nltk.corpus.stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

limit_per_tag = 40

for intent in intents['intents']:
    augmented_sentences_per_tag = 0
    for example in intent['patterns']:
        tokens = nltk.word_tokenize(example.lower())
        filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords and token.isalpha()]
        if filtered_tokens:
            text_data.append(' '.join(filtered_tokens))
            labels.append(intent['tag'])

            augmented_sentences = synonym_replacement(filtered_tokens, limit_per_tag - augmented_sentences_per_tag)
            for augmented_sentence in augmented_sentences:
                text_data.append(augmented_sentence)
                labels.append(intent['tag'])
                augmented_sentences_per_tag += 1
                if augmented_sentences_per_tag >= limit_per_tag:
                    break

print(len(text_data))
print(len(labels))

3205
3205


#TF-IDF Vectorization
This code snippet uses TfidfVectorizer from scikit-learn to convert text_data into TF-IDF matrix X and assigns corresponding labels to y. TF-IDF vectorization is essential for transforming text data into numerical format for machine learning models.

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_data)
y = labels

#Model Selection with Grid Search
This function, find_best_model, implements model selection by training multiple classifiers using grid search with cross-validation. It splits the data into training and testing sets, iterates through different classifiers with specified hyperparameter grids, and prints the accuracy scores with their best parameters. Finally, it selects the best-performing model based on the highest accuracy score on the test set and fits the best model to the full training data before returning it.

In [ ]:
def find_best_model(X, y, test_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=100)
    models = [
        ('Logistic Regression', LogisticRegression(), {
            'penalty': ['l2'],
            'C': [0.1, 1.0, 10.0],
            'solver': ['liblinear'],
            'max_iter': [100, 1000, 10000]
        }),
        ('Multinomial Naive Bayes', MultinomialNB(), {'alpha': [0.1, 0.5, 1.0]}),
        ('Linear SVC', LinearSVC(), {
            'penalty': ['l2'],
            'loss': ['hinge', 'squared_hinge'],
            'C': [0.1, 1, 10],
            'max_iter': [100, 1000, 10000]
        }),
        ('Decision Tree', DecisionTreeClassifier(), {
            'max_depth': [5, 10, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'criterion': ['gini', 'entropy']
        }),
        ('Random Forest', RandomForestClassifier(), {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        })
    ]
    for name, model, param_grid in models:
        grid = GridSearchCV(model, param_grid, cv=3, n_jobs=-1)
        grid.fit(X_train, y_train)
        y_pred = grid.predict(X_test)
        score = accuracy_score(y_test, y_pred)
        print(f'{name}: {score:.4f} (best parameters: {grid.best_params_})')

    best_model = max(models, key=lambda x: GridSearchCV(x[1], x[2], cv=3, n_jobs=-1).fit(X_train, y_train).score(X_test, y_test))
    print(f'\nBest model: {best_model[0]}')

    # Fit the best model to the full training data
    best_model[1].fit(X, y)
    return best_model[1]

In [ ]:
best_model = find_best_model(X, y)

Logistic Regression: 0.9064 (best parameters: {'C': 10.0, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'})
Multinomial Naive Bayes: 0.8986 (best parameters: {'alpha': 0.1})
Linear SVC: 0.9111 (best parameters: {'C': 10, 'loss': 'hinge', 'max_iter': 1000, 'penalty': 'l2'})
Decision Tree: 0.9111 (best parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2})
Random Forest: 0.9189 (best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300})

Best model: Random Forest


#Chatbot Response Generation
This function, chatbot_response, generates a response for the user input using the trained best_model. It first transforms the user input into a TF-IDF vector using the vectorizer object. Then, it predicts the intent of the input using the best_model and selects a random response from the corresponding intent defined in the intents dictionary. Finally, it returns the selected response.

In [ ]:
def chatbot_response(user_input):
    input_text = vectorizer.transform([user_input])
    predicted_intent = best_model.predict(input_text)[0]

    for intent in intents['intents']:
        if intent['tag'] == predicted_intent:
            response = random.choice(intent['responses'])
            break

    return response

#Chatbot Interaction Loop
This code initiates an interaction loop with the chatbot, prompting the user for input. If the user inputs "quit", the loop terminates, and the chatbot signs off. Otherwise, it generates a response using the chatbot_response function based on the user input and prints it.

In [ ]:
print('Hello! I am a chatbot. How can I help you today? Type "quit" to exit.')
while True:
    user_input = input('> ')
    if user_input.lower() == 'quit':
        print("Signing Off !!")
        break
    response = chatbot_response(user_input)
    print(response)

Hello! I am a chatbot. How can I help you today? Type "quit" to exit.
Hello!
I was created by Veludandi Harini.
You can call me harini. I'm a Chatbot.
The college is located at bollikunta, Warangal district. You can find the location on Google Maps.
Hello!
The college offers courses in Artificial Intellegence and Machine Learning, Cyber Security, Data Science, Artifficial Intellegence and Data Science, Information Technology, Computer Engineering, Mechanical Engineering, Chemical Engineering, and Civil Engineering.
For detailed fee information, please visit our college website.
The college is located at bollikunta, Warangal district. You can find the location on Google Maps.
Yes, College provides hostel facilities. For detailed information about the hostel, please visit our college website.
I was created by Veludandi Harini.
You have to write Eamcet exam as a part of admission process. For detailed admission information, please visit our college website.
Yes, the college has a library.